In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor,  BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import altair as alt
import datetime as dt
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf  # for data
from pandas_datareader import data as pdr
yf.pdr_override()

In [22]:
def read_data(stock='TSLA', period = "60d", interval = "2m"):
    '''
    reading the data
    '''


    df = pdr.get_data_yahoo(stock,period = period,

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = interval,)


    return df

In [23]:
df = read_data(stock='TSLA').reset_index()
df.head()

[*********************100%***********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2020-08-31 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-09-28 09:30:00-04:00,424.619995,427.000000,422.000000,425.429993,425.429993,3037163.0
2,2020-09-28 09:32:00-04:00,425.470001,428.079987,424.605011,426.649994,426.649994,875668.0
3,2020-09-28 09:34:00-04:00,426.899994,426.899994,423.451904,423.695007,423.695007,667923.0
4,2020-09-28 09:36:00-04:00,423.869995,425.290009,421.200012,422.690002,422.690002,728853.0


> The most old data located at the top

In [4]:
df.tail()

,Datetime,Open,High,Low,Close,Adj Close,Volume
6537,2020-11-12 12:58:00-05:00,413.589996,414.500000,413.559998,414.364990,414.364990,77142.0
6538,2020-11-12 13:00:00-05:00,414.299988,414.440002,413.160004,413.226105,413.226105,70579.0
6539,2020-11-12 13:02:00-05:00,413.320007,414.500000,413.299988,414.480011,414.480011,49894.0
6540,2020-11-12 13:04:00-05:00,414.399994,414.500000,413.725006,413.989899,413.989899,56497.0
6541,2020-11-12 13:06:27-05:00,413.924988,413.924988,413.924988,413.924988,413.924988,0.0


> The most recent data located at the bottom

### Data Size

In [5]:
len(read_data(stock='TSLA', period = "7d", interval = "1m"))


[*********************100%***********************]  1 of 1 completed


2551

In [6]:
len(read_data(stock='TSLA', period = "60d", interval = "2m"))



[*********************100%***********************]  1 of 1 completed


6542

In [7]:
len(read_data(stock='TSLA', period = "60d", interval = "5m"))


[*********************100%***********************]  1 of 1 completed


4644

Sample size is very small for 5 minutes. We can't go beyond 7 days. One strategy to solve this problem is to get data everyday and then merge it with the previous data

> In term of data size, the most promsing is the 2 minutes data unless we have old data from somewhere else

In [8]:
df_sub1 = df[['Datetime','Open','Close','Volume']]


In [9]:
df_sub2 = df_sub1.copy()

In [10]:
# df_sub2['close_previous'] = df_sub2['Close'].shift(1)
# df_sub2.tail()

> shift(1) gives the previous time interval data, which is available

In [11]:
# 1 change in price, open now/closed previously

In [12]:
df_sub2['f01'] = ((df_sub2['Open']/df_sub2['Close'].shift(1))-1)*100
df_sub2.head()


,Datetime,Open,Close,Volume,f01
0,2020-08-31 09:30:00-04:00,NaN,NaN,NaN,NaN
1,2020-09-28 09:30:00-04:00,424.619995,425.429993,3037163.0,NaN
2,2020-09-28 09:32:00-04:00,425.470001,426.649994,875668.0,0.009404
3,2020-09-28 09:34:00-04:00,426.899994,423.695007,667923.0,0.058596
4,2020-09-28 09:36:00-04:00,423.869995,422.690002,728853.0,0.041300


In [13]:
df_sub2.tail()

,Datetime,Open,Close,Volume,f01
6537,2020-11-12 12:58:00-05:00,413.589996,414.364990,77142.0,0.021765
6538,2020-11-12 13:00:00-05:00,414.299988,413.226105,70579.0,-0.015687
6539,2020-11-12 13:02:00-05:00,413.320007,414.480011,49894.0,0.022724
6540,2020-11-12 13:04:00-05:00,414.399994,413.989899,56497.0,-0.019305
6541,2020-11-12 13:06:27-05:00,413.924988,413.924988,0.0,-0.015679


In [14]:
df_sub2['Datetime'] = pd.to_datetime(df_sub2['Datetime'])
df_sub2['Datetime']=df_sub2['Datetime'].map(dt.datetime.toordinal)
df_sub2 = df_sub2.dropna()

In [15]:
y = df_sub2['Close']
y

2       426.649994
3       423.695007
4       422.690002
5       424.265015
6       424.234985
           ...    
6537    414.364990
6538    413.226105
6539    414.480011
6540    413.989899
6541    413.924988
Name: Close, Length: 6540, dtype: float64

In [16]:
X = df_sub2.drop(columns =['Close','Volume'])
X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)

In [17]:
old = df_sub2.tail(1).loc[:,'Open'].to_numpy()[0]
old

413.92498779296875

In [18]:
df_sub2.tail(1)

,Datetime,Open,Close,Volume,f01
6541,737741,413.924988,413.924988,0.0,-0.015679


In [19]:
numeric_features = list(X_train.columns)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
   ])
# now a taransformer for the categorical features
categorical_features = ['status_lower','status_upper']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
# creating a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

ridge_model = RandomForestRegressor()
# include the preprocessor and the model in one pipeline.
# Now we have a full prediction pipeline.
reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('Regressor', ridge_model)])


param_grid = {
    'Regressor__n_estimators':[100],
}

search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

# fitting the model
search.fit(X_train, y_train)
print(search.score(X_train, y_train))
y_pred = search.predict(X_train)
print('training error',np.sqrt(mean_squared_error(y_train,y_pred)))
print("model score: %.3f" % search.score(X_test, y_test))
y_pred1 = search.predict(X_test)
print('test error',np.sqrt(mean_squared_error(y_test,y_pred1)))


# printing the first parameter
#rint(search.best_params_)
# print("model score: %.3f" % search.score(X_test, y_test))
# X_train = X[:-1]
# y_train = y[:-1]
# X_test = X[-1:]
# y_test = y[-1:]
# search.fit(X_train,y_train)
# y_pred = search.predict(X_train)
# print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
# y_test = search.predict(X_test)[0]
# diff = y_test - old

# print('old price = ',old)
# print('predicted = ',y_test)
# print('difference = ', diff)




0.9996041351962721
training error 0.30086800675529596
model score: 0.907
test error 3.6621883052450594


In [21]:


df = dfx.copy()

# for now the following do not add any predictiv power
#df['day_of_the_week'] = df['Datetime'].dt.dayofweek
#df['day_of_month'] = df['Datetime'].apply(lambda x:int(str(x)[8:10]))


print(df.tail(1)['Datetime'])
old = df.tail(1).loc[:,'Open'].to_numpy()[0]

df['Datetime'] = pd.to_datetime(df['Datetime'])
df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')

df['High'] = df['High'].shift(1)
df['Low'] = df['Low'].shift(1)
df['Volume'] = df['Volume'].shift(1)
df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
#df['f01'] = df['Close']/df['Open']-1


df = df.dropna()
df = df.drop(columns = ['Close'])
y = df['Adj Close']
X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
    # first transformer for the numeric features
print(X_train)
numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
   ])
# now a taransformer for the categorical features
categorical_features = ['status_lower','status_upper']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
# creating a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

ridge_model = Ridge()
# include the preprocessor and the model in one pipeline.
# Now we have a full prediction pipeline.
reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('Regressor', ridge_model)])

# finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
param_grid = {
    'Regressor__alpha':[0.1,0.25,0.4],
}



search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

# fitting the model
search.fit(X_train, y_train)

# printing the first parameter
#rint(search.best_params_)
print("model score: %.3f" % search.score(X_test, y_test))
X_train = X[:-1]
y_train = y[:-1]
X_test = X[-1:]
y_test = y[-1:]
search.fit(X_train,y_train)
y_pred = search.predict(X_train)
print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
y_test = search.predict(X_test)[0]
diff = y_test - old

print('old price = ',old)
print('predicted = ',y_test)
print('difference = ', diff)








NameError: name 'dfx' is not defined